In [1]:
from mxnet import nd, gluon, autograd

In [2]:
def transform(x, y):
    return x.astype('float32')/255, y.astype('float32')


In [6]:
mnistTrain = gluon.data.vision.FashionMNIST(root='./workdir/mnist/', train=True, transform=transform)
mnistTest = gluon.data.vision.FashionMNIST(root='./workdir/mnist/', train=False, transform=transform)

In [7]:
batchSize = 256
trainData = gluon.data.DataLoader(mnistTrain, batchSize, shuffle=True)
testData = gluon.data.DataLoader(mnistTest, batchSize, shuffle=False)

In [8]:
weight_scale = .01

# output channels = 20, kernel = (5,5)
w1 = nd.random_normal(shape=(20,1,5,5), scale=weight_scale)
b1 = nd.zeros(w1.shape[0])

# output channels = 50, kernel = (3,3)
w2 = nd.random_normal(shape=(50,20,3,3), scale=weight_scale)
b2 = nd.zeros(w2.shape[0])

# output dim = 128
w3 = nd.random_normal(shape=(1250, 128), scale=weight_scale)
b3 = nd.zeros(w3.shape[1])

# output dim = 10
w4 = nd.random_normal(shape=(w3.shape[1], 10), scale=weight_scale)
b4 = nd.zeros(w4.shape[1])

params = [w1, b1, w2, b2, w3, b3, w4, b4]
for param in params:
    param.attach_grad()

In [9]:
def net(x, verbose=False):
    # 第一层卷积
    h1Conv = nd.Convolution(
        data=x, weight=w1, bias=b1, kernel=w1.shape[2:], num_filter=w1.shape[0])
    h1Activation = nd.relu(h1Conv)
    h1 = nd.Pooling(
        data=h1Activation, pool_type="max", kernel=(2,2), stride=(2,2))
    # 第二层卷积
    h2Conv = nd.Convolution(
        data=h1, weight=w2, bias=b2, kernel=w2.shape[2:], num_filter=w2.shape[0])
    h2Activation = nd.relu(h2Conv)
    h2 = nd.Pooling(data=h2Activation, pool_type="max", kernel=(2,2), stride=(2,2))
    h2 = nd.flatten(h2)
    # 第一层全连接
    h3Dense = nd.dot(h2, w3) + b3
    h3 = nd.relu(h3Dense)
    # 第二层全连接
    h4Dense = nd.dot(h3, w4) + b4
    if verbose:
        print('1st conv block:', h1.shape)
        print('2nd conv block:', h2.shape)
        print('1st dense:', h3.shape)
        print('2nd dense:', h4Dense.shape)
        print('output:', h4Dense)
    return h4Dense

In [10]:
for x,y in trainData:
    print(x.shape)
    break

(256, 28, 28, 1)


In [11]:
for x,y in trainData:
    print(nd.transpose(x,(0,3,1,2)).shape)
    break

(256, 1, 28, 28)


In [12]:
for x,y in trainData:
    net(nd.transpose(x,(0,3,1,2)), verbose=True)
    break

1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ -7.06006176e-05  -6.22040316e-05  -6.91514288e-05 ...,   1.03286766e-04
   -5.67714960e-05   9.43985287e-05]
 [ -9.22994150e-05  -5.94710473e-05  -5.80573687e-05 ...,   1.18989985e-04
   -8.97821665e-05   7.43785640e-05]
 [ -6.80073390e-06  -6.95893104e-05  -6.82389145e-05 ...,   7.63719872e-05
   -1.05617866e-04   5.66465314e-05]
 ..., 
 [ -6.83608814e-05  -6.43983803e-05  -3.07222290e-05 ...,   1.21976853e-04
   -4.72249667e-05   7.49007013e-05]
 [ -4.73910732e-05  -2.53048092e-05  -2.56299063e-05 ...,   8.26592004e-05
   -3.33367534e-05   5.73266225e-05]
 [ -1.79199906e-05  -2.86292307e-05  -5.18683846e-05 ...,   5.72593308e-05
   -8.70687800e-05   9.39384554e-05]]
<NDArray 256x10 @cpu(0)>


In [13]:
def SGD(params, eta):
    for param in params:
        param[:] = param - eta * param.grad

In [14]:
softmaxCrossEntropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [15]:
def accuracy(yhat, y):
    return nd.mean(yhat.argmax(axis=1)==y).asscalar()

In [16]:
def evaluateAcc(dataIterator, net):
    acc = 0.
    for x, y in dataIterator:
        yhat = net(nd.transpose(x,(0,3,1,2)))
        acc += accuracy(yhat, y)
    return acc / len(dataIterator)

In [17]:
evaluateAcc(testData, net)

0.082714843750000003

In [18]:
eta = 0.1
epochs = 5

In [21]:
for epoch in range(epochs):
    trainLoss = 0.
    trainAcc = 0.
    for x, y in trainData:
        with autograd.record():
            yhat = net(nd.transpose(x,(0,3,1,2)))
            loss = softmaxCrossEntropy(yhat, y)
        loss.backward()
        SGD(params, eta/batchSize)
        trainLoss += nd.mean(loss).asscalar()
        trainAcc += accuracy(yhat, y)
        
    testAcc = evaluateAcc(testData, net)
    print('Epoch %d. Loss: %f, Train acc: %f, Test acc %f' % (
        epoch, trainLoss/len(trainData), trainAcc/len(trainData), testAcc))

Epoch 0. Loss: 0.206720, Train acc: 0.923050, Test acc 0.909961
Epoch 1. Loss: 0.204776, Train acc: 0.922750, Test acc 0.908691
Epoch 2. Loss: 0.201560, Train acc: 0.924169, Test acc 0.911230
Epoch 3. Loss: 0.198631, Train acc: 0.925521, Test acc 0.908008
Epoch 4. Loss: 0.196909, Train acc: 0.926989, Test acc 0.909375
